# Imports

In [8]:
import re
from pathlib import Path
import os
import pandas as pd

# Start from current path and walk up until we find a known file
curr_path = Path().resolve()
marker_file = "requirements.txt"  # or any other file you know exists in your project root

while not (curr_path / marker_file).exists() and curr_path != curr_path.parent:
    curr_path = curr_path.parent

# Set working directory if found
if (curr_path / marker_file).exists():
    os.chdir(curr_path)
    print("Now working in:", Path.cwd())
else:
    raise RuntimeError(f"Project root with {marker_file} not found.")

Now working in: C:\Users\yonat\CodeProjects\Israel-Palestine-Political-Affiliation-Text-Classification


# Cleaning the Original Files

In [ ]:
# # ---- config ----
# DATA_DIR = Path("Data/p-stance")
# FILES    = {"TRAIN": "train_trump.csv", "VAL": "val_trump.csv", "TEST": "test_trump.csv"}
# OUT_CSV  = DATA_DIR / "pstance_trump_clean.csv"
# SOURCE   = "P Stance"

# # remove links, mentions, and entire hashtag tokens
# URL_TOKEN     = re.compile(r"(https?://\S+|www\.\S+)", flags=re.IGNORECASE)
# MENTION_TOKEN = re.compile(r"@\w+")
# HASHTAG_TOKEN = re.compile(r"(?<!\w)#\w+")
# EMOJI_TOKEN   = re.compile(r"[\U0001F1E6-\U0001F1FF"
#                            r"\U0001F300-\U0001F5FF"
#                            r"\U0001F600-\U0001F64F"
#                            r"\U0001F680-\U0001F6FF"
#                            r"\U0001F700-\U0001F77F"
#                            r"\U0001F780-\U0001F7FF"
#                            r"\U0001F800-\U0001F8FF"
#                            r"\U0001F900-\U0001F9FF"
#                            r"\U0001FA00-\U0001FA6F"
#                            r"\u2600-\u27BF]", flags=re.UNICODE)
# MULTI_SPACE   = re.compile(r"\s+")

# def clean_text(s: str) -> str:
#     if not isinstance(s, str):
#         return ""
#     s = URL_TOKEN.sub(" ", s)         # drop URLs (gif/image-only posts become empty)
#     s = MENTION_TOKEN.sub(" ", s)     # drop @handles
#     s = HASHTAG_TOKEN.sub(" ", s)     # drop full hashtags
#     s = EMOJI_TOKEN.sub(" ", s)       # drop emojis/symbols
#     s = s.replace("\u200b", " ")
#     s = MULTI_SPACE.sub(" ", s).strip()
#     return s

# # ---- map to your three-label scheme ----
# label_map = {
#     "favor":   "Pro-Trump",
#     "favour":  "Pro-Trump",
#     "against": "Against-Trump",
#     "neutral": "Undefined",
#     "neither": "Undefined",
#     "none":    "Undefined",
# }

# def load_split(path: Path, split_name: str) -> pd.DataFrame:
#     df = pd.read_csv(path)
#     missing = {"Tweet", "Stance"} - set(df.columns)
#     if missing:
#         raise ValueError(f"Missing columns in {path.name}: {missing}")

#     out = pd.DataFrame({
#         "self_text": df["Tweet"].astype(str).map(clean_text),
#         "real_label": df["Stance"].astype(str).str.lower().map(lambda x: label_map.get(x, x)),
#         "source": SOURCE,
#         "split": split_name.upper(),
#     })
#     # drop rows that became empty after cleaning
#     out = out[out["self_text"].str.len() > 0].reset_index(drop=True)
#     return out

# # ---- run ----
# frames = [load_split(DATA_DIR / fname, split) for split, fname in FILES.items()]
# df_all = pd.concat(frames, ignore_index=True)

# # prepend a local ID
# df_all.insert(0, "local_comment_id", range(1, len(df_all) + 1))

# # enforce final column order and save
# df_all = df_all[["local_comment_id", "self_text", "source", "real_label", "split"]]
# df_all.to_csv(OUT_CSV, index=False)

# print(f"Saved: {OUT_CSV} ({len(df_all):,} rows)")
# print(df_all.groupby(["split", "real_label"]).size().unstack(fill_value=0))

Saved: Data\p-stance\pstance_trump_clean.csv (7,953 rows)
real_label  Against-Trump  Pro-Trump
split                               
TEST                  425        352
TRAIN                3425       2937
VAL                   440        374


At this point we manually added the 'Undefined' comments from the other dataset, adding comment_id to all (fabricated, p=stance has no comment IDs), and re-labels the dataset using an LLM. Prompt is available in `Config.data_tagging_config.py`

# Training Classifier

Dataset is assumed to have a new column, `label`, where the `TEST` samples have the real label and the `TRAIN`, `VAL` labels has the LLM created label.

In [ ]:
# ---- config ----
DATA_DIR = Path("Data/p-stance")
OUT_CSV  = DATA_DIR / "pstance_trump_final.csv"

df = pd.read_csv(OUT_CSV)
df.head()

,local_comment_id,self_text,source,real_label,split
0,1,You COWARD... you should have been at THE WHIT...,P Stance,Against-Trump,TRAIN
1,2,Cool. I trust he requested the same records fo...,P Stance,Against-Trump,TRAIN
2,3,Off your meds again Donnie? You are surrounded...,P Stance,Against-Trump,TRAIN
3,4,That has to happen anyway. But his profits wil...,P Stance,Against-Trump,TRAIN
4,5,This is so stupid. Unmasking lets them know wh...,P Stance,Against-Trump,TRAIN
